# Analysis of the Global Innovation Index (GII) by WIPO

Summary: https://www.wipo.int/web-publications/global-innovation-index-2024/en/index.html  
Dataset: https://www.wipo.int/publications/de/details.jsp?id=4756&plang=DE


## Download & Preprocess Data

In [20]:
import wget
import os.path
import pandas as pd

filename = "../data/GII_2024.xlsx"
if not os.path.isfile(filename):
    url = "https://www.wipo.int/edocs/pubdocs/en/wipo-pub-2000-2024-tech1.xlsx"
    filename = wget.download(url, out=filename)

dfs = pd.read_excel(filename, sheet_name=None)

gii_data = dfs["Data"]
display(gii_data)
gii_economies = dfs["Economies"]
display(gii_economies)

joined = gii_data.merge(gii_economies, on="ISO3", suffixes=["", "_joined_r"]).drop(columns="ECONOMY_NAME_joined_r")
display(joined)

gii_data_eu = joined[joined["REG_UN_CODE"] == "EUR"].drop(columns=["REG_UN", "REG_UN_CODE"])
display(gii_data_eu)

,ISO3,ECONOMY_NAME,NUM,NAME,DATAYR,VALUE_SCREEN,SCORE,RANK,SW_OVERALL,SW_INCGRP,OUTDATED,DMC
0,ALB,Albania,NaN,Global Innovation Index,NaN,NaN,24.537930,84.0,NaN,NaN,NaN,NaN
1,ALB,Albania,IN,Innovation inputs,NaN,NaN,35.043195,66.0,NaN,NaN,NaN,NaN
2,ALB,Albania,IN.1,Institutions,NaN,NaN,50.321760,60.0,NaN,NaN,NaN,0.0
3,ALB,Albania,IN.1.1,Institutional environment,NaN,NaN,55.174345,60.0,NaN,NaN,NaN,0.0
4,ALB,Albania,IN.1.1.1,Operational stability for businesses*,2023.0,2.025000,64.666667,61.0,NaN,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
14492,ZWE,Zimbabwe,OUT.7.2.4,"Creative goods exports, % total trade",2022.0,0.121097,1.516072,90.0,NaN,NaN,0.0,NaN
14493,ZWE,Zimbabwe,OUT.7.3,Online creativity,NaN,NaN,16.368272,110.0,NaN,NaN,NaN,0.0
14494,ZWE,Zimbabwe,OUT.7.3.1,Top-level domains (TLDs)/th pop. 15–69,2023.0,6.200259,0.843852,99.0,NaN,NaN,0.0,NaN
14495,ZWE,Zimbabwe,OUT.7.3.2,GitHub commits/mn pop. 15–69,2023.0,53069.765227,0.955263,115.0,NaN,NaN,0.0,NaN


,ISO3,ECONOMY_NAME,INCOME,REG_UN,REG_UN_CODE,POP,PPPGDP,PPPPC
0,ALB,Albania,UM,Europe,EUR,2811.65,55.928,19566.01
1,DZA,Algeria,LM,Northern Africa and Western Asia,NAWA,46164.22,628.990,13681.62
2,AGO,Angola,LM,Sub-Saharan Africa,SSA,36749.91,260.323,7077.30
3,ARG,Argentina,UM,Latin America and the Caribbean,LCN,45538.40,1239.520,26506.09
4,ARM,Armenia,UM,Northern Africa and Western Asia,NAWA,2943.38,58.497,19744.71
...,...,...,...,...,...,...,...,...
128,URY,Uruguay,HI,Latin America and the Caribbean,LCN,3388.08,103.372,28983.79
129,UZB,Uzbekistan,LM,Central and Southern Asia,CSA,35652.30,371.646,10316.37
130,VNM,Viet Nam,LM,"South East Asia, East Asia, and Oceania",SEAO,100352.19,1434.210,14285.09
131,ZMB,Zambia,LM,Sub-Saharan Africa,SSA,20723.97,83.687,4068.44


,ISO3,ECONOMY_NAME,NUM,NAME,DATAYR,VALUE_SCREEN,SCORE,RANK,SW_OVERALL,SW_INCGRP,OUTDATED,DMC,INCOME,REG_UN,REG_UN_CODE,POP,PPPGDP,PPPPC
0,ALB,Albania,NaN,Global Innovation Index,NaN,NaN,24.537930,84.0,NaN,NaN,NaN,NaN,UM,Europe,EUR,2811.65,55.928,19566.01
1,ALB,Albania,IN,Innovation inputs,NaN,NaN,35.043195,66.0,NaN,NaN,NaN,NaN,UM,Europe,EUR,2811.65,55.928,19566.01
2,ALB,Albania,IN.1,Institutions,NaN,NaN,50.321760,60.0,NaN,NaN,NaN,0.0,UM,Europe,EUR,2811.65,55.928,19566.01
3,ALB,Albania,IN.1.1,Institutional environment,NaN,NaN,55.174345,60.0,NaN,NaN,NaN,0.0,UM,Europe,EUR,2811.65,55.928,19566.01
4,ALB,Albania,IN.1.1.1,Operational stability for businesses*,2023.0,2.025000,64.666667,61.0,NaN,NaN,0.0,NaN,UM,Europe,EUR,2811.65,55.928,19566.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14492,ZWE,Zimbabwe,OUT.7.2.4,"Creative goods exports, % total trade",2022.0,0.121097,1.516072,90.0,NaN,NaN,0.0,NaN,LM,Sub-Saharan Africa,SSA,16340.84,44.448,2749.78
14493,ZWE,Zimbabwe,OUT.7.3,Online creativity,NaN,NaN,16.368272,110.0,NaN,NaN,NaN,0.0,LM,Sub-Saharan Africa,SSA,16340.84,44.448,2749.78
14494,ZWE,Zimbabwe,OUT.7.3.1,Top-level domains (TLDs)/th pop. 15–69,2023.0,6.200259,0.843852,99.0,NaN,NaN,0.0,NaN,LM,Sub-Saharan Africa,SSA,16340.84,44.448,2749.78
14495,ZWE,Zimbabwe,OUT.7.3.2,GitHub commits/mn pop. 15–69,2023.0,53069.765227,0.955263,115.0,NaN,NaN,0.0,NaN,LM,Sub-Saharan Africa,SSA,16340.84,44.448,2749.78


,ISO3,ECONOMY_NAME,NUM,NAME,DATAYR,VALUE_SCREEN,SCORE,RANK,SW_OVERALL,SW_INCGRP,OUTDATED,DMC,INCOME,POP,PPPGDP,PPPPC
0,ALB,Albania,NaN,Global Innovation Index,NaN,NaN,24.537930,84.0,NaN,NaN,NaN,NaN,UM,2811.65,55.928,19566.01
1,ALB,Albania,IN,Innovation inputs,NaN,NaN,35.043195,66.0,NaN,NaN,NaN,NaN,UM,2811.65,55.928,19566.01
2,ALB,Albania,IN.1,Institutions,NaN,NaN,50.321760,60.0,NaN,NaN,NaN,0.0,UM,2811.65,55.928,19566.01
3,ALB,Albania,IN.1.1,Institutional environment,NaN,NaN,55.174345,60.0,NaN,NaN,NaN,0.0,UM,2811.65,55.928,19566.01
4,ALB,Albania,IN.1.1.1,Operational stability for businesses*,2023.0,2.025000e+00,64.666667,61.0,NaN,NaN,0.0,NaN,UM,2811.65,55.928,19566.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13729,GBR,United Kingdom,OUT.7.2.4,"Creative goods exports, % total trade",2022.0,1.918612e+00,24.047970,27.0,NaN,NaN,0.0,NaN,HI,68682.99,3871.790,56835.72
13730,GBR,United Kingdom,OUT.7.3,Online creativity,NaN,NaN,63.301512,12.0,NaN,NaN,NaN,0.0,HI,68682.99,3871.790,56835.72
13731,GBR,United Kingdom,OUT.7.3.1,Top-level domains (TLDs)/th pop. 15–69,2023.0,4.062317e+02,56.253910,9.0,NaN,NaN,0.0,NaN,HI,68682.99,3871.790,56835.72
13732,GBR,United Kingdom,OUT.7.3.2,GitHub commits/mn pop. 15–69,2023.0,3.072424e+06,58.844818,18.0,NaN,NaN,0.0,NaN,HI,68682.99,3871.790,56835.72


EU countries sorted by innovation level

In [61]:
pure_rank = gii_data_eu[gii_data_eu["NUM"].isnull()]
display_data = pure_rank.sort_values("RANK")[["RANK", "ECONOMY_NAME", "SCORE"]]
display_data.rename(columns={"RANK": "GLOBAL_RANK"}, inplace=True)
display_data.index = range(1, len(display_data) + 1)

display(display_data)

,GLOBAL_RANK,ECONOMY_NAME,SCORE
1,1.0,Switzerland,67.473082
2,2.0,Sweden,64.524365
3,5.0,United Kingdom,61.049318
4,7.0,Finland,59.436931
5,8.0,Netherlands (Kingdom of the),58.833036
6,9.0,Germany,58.082837
7,10.0,Denmark,57.114019
8,12.0,France,55.413323
9,16.0,Estonia,52.307935
10,17.0,Austria,50.276568
